In [1]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy

# random_hamiltonian

In [2]:
ttn = ptn.random_big_ttns_two_root_children(mode= ptn.RandomTTNSMode.DIFFVIRT)

In [46]:
X , Y , Z = ptn.pauli_matrices()
possible_operators = [X, X@Y, Z@X, Z, Y, X@Y , Z@Z, Y@X, Z@Y ] 

H = ptn.random_hamiltonian(num_of_terms= 5,
                           possible_operators= possible_operators,
                           tree= ttn,
                           strength = (-2,5),
                           num_sites = (1,4))
H = H.pad_with_identities(ttn , symbolic= False)

H_op = H.to_matrix(ttn) # order = list(ttn.nodes.keys())
print(H_op.operator.shape , H_op.node_identifiers)
H_tn = H.to_tensor(ttn)
print(H_tn.operator.shape , H_tn.node_identifiers)


(256, 256) ['site0', 'site1', 'site2', 'site3', 'site4', 'site5', 'site6', 'site7']
(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2) ['site0', 'site1', 'site2', 'site3', 'site4', 'site5', 'site6', 'site7']


In [40]:
possible_operators = ["X", "Y", "Z"]
H = ptn.random_hamiltonian(num_of_terms= 5,
                           possible_operators= possible_operators,
                           tree= ttn,
                           strength = (-2,5),
                           num_sites = (1,4),
                           conversion_dict= {"X": X, "Y": Y, "Z": Z , "I2": np.eye(2)})
H = H.pad_with_identities(ttn , symbolic= True)

random_ttno = ptn.TTNO.from_hamiltonian(H, ttn)

# random_tensor_product

In [47]:
possible_operators = [X, X@Y, Z@X, Z, Y, X@Y , Z@Z, Y@X, Z@Y ] 
random_numeric_tp = ptn.random_tensor_product(ref_tree= ttn,
                                              num_operators= 5,
                                              possible_operators= possible_operators,
                                              factor= 1)   
H = ptn.Hamiltonian(random_numeric_tp)
H = H.pad_with_identities(ttn)

In [23]:
possible_operators = ["X", "Y", "Z"]
random_symbolic_tp = ptn.random_tensor_product(ref_tree= ttn,
                                              num_operators= 3,
                                              possible_operators= possible_operators,
                                              factor= 1) 

tp_list_symbolic = [ptn.random_tensor_product(ttn,3,["A","B","C","D"]) for _ in range(5)] 
conversion_dictionary = {"A": ptn.random_hermitian_matrix(2),
                         "B": ptn.random_hermitian_matrix(2),
                         "C": ptn.random_hermitian_matrix(2),
                         "D": ptn.random_hermitian_matrix(2),
                         "I2": np.eye(2)}

H = ptn.Hamiltonian(tp_list_symbolic, conversion_dictionary=conversion_dictionary)
H = H.pad_with_identities(ttn)
random_ttno = ptn.TTNO.from_hamiltonian(H, ttn)   

In [ ]:
def Heisenberg_ham(J, ttn):

    paulis = ["X", "Y", "Z"]
    pauli_ops = ptn.pauli_matrices()
    terms = []
    conversion_dict = {"I2" : np.eye(2)}
    for i, operator in enumerate(pauli_ops):
        conversion_dict[paulis[i]] = operator
        conversion_dict["-J" + paulis[i]] = J * operator

    for identifier, node in ttn.nodes.items():
        for child in node.children:
            for pauli in paulis:
                terms.append(ptn.TensorProduct({identifier: "-J" + pauli,
                                        child: pauli}))
    return ptn.Hamiltonian(terms, conversion_dict) 

def XY_ham(J, h, ttn1):
    pauli_ops = ptn.pauli_matrices()
    terms = []
    conversion_dict = {"I2": np.eye(2)}
    
    # X-X and Y-Y interactions
    for i, pauli in enumerate(["X", "Y"]):
        conversion_dict[pauli] = pauli_ops[i]
        conversion_dict[f"-J{pauli}"] = J * pauli_ops[i]
    
    # Transverse field (usually in Z direction for XY model)
    conversion_dict["-hZ"] = h * pauli_ops[2]

    # X-X and Y-Y interactions between connected sites
    for identifier, node in ttn1.nodes.items():
        for child in node.children:
            terms.append(ptn.TensorProduct({identifier: "-JX", child: "X"}))
            terms.append(ptn.TensorProduct({identifier: "-JY", child: "Y"}))
    
    # Transverse field terms for each site
    for identifier in ttn1.nodes:
        terms.append(ptn.TensorProduct({identifier: "-hZ"}))

    return ptn.Hamiltonian(terms, conversion_dict)

def ZZ_ham(J, h, ttn1):
    pauli_ops = ptn.pauli_matrices()
    terms = []
    conversion_dict = {"I2": np.eye(2)}
    
    # Z-Z interaction
    conversion_dict["Z"] = pauli_ops[2]
    conversion_dict["-JZ"] = J * pauli_ops[2]
    
    # Transverse field
    conversion_dict["-hX"] = h * pauli_ops[0]

    # Z-Z interactions between connected sites
    for identifier, node in ttn1.nodes.items():
        for child in node.children:
            terms.append(ptn.TensorProduct({identifier: "-JZ", child: "Z"}))
    
    # Transverse field terms for each site
    for identifier in ttn1.nodes:
        terms.append(ptn.TensorProduct({identifier: "-hX"}))

    return ptn.Hamiltonian(terms, conversion_dict)

def ZZ_ham(J, h, ttn1):
    pauli_ops = ptn.pauli_matrices()
    terms = []
    conversion_dict = {"I2": np.eye(2)}
    
    # Z-Z interaction
    conversion_dict["Z"] = pauli_ops[2]
    conversion_dict["-JZ"] = J * pauli_ops[2]
    
    # Transverse field
    conversion_dict["-hX"] = h * pauli_ops[0]

    # Z-Z interactions between connected sites
    for identifier, node in ttn1.nodes.items():
        for child in node.children:
            terms.append(ptn.TensorProduct({identifier: "-JZ", child: "Z"}))
    
    # Transverse field terms for each site
    for identifier in ttn1.nodes:
        terms.append(ptn.TensorProduct({identifier: "-hX"}))

    return ptn.Hamiltonian(terms, conversion_dict)
